In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
dir = 'emotion'
filepaths = []
labels = []

folds = os.listdir(dir)
for fold in folds:
    foldpath = os.path.join(dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

Fseries = pd.Series(filepaths, name= 'image')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
df.head()

In [ ]:
import matplotlib.image as mpimg

img_path = filepaths[25]
img = mpimg.imread(img_path)
plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.axis('off') 
label = labels[25]
plt.title(f'Label: {label}')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df,  train_size= 0.6, shuffle= True, random_state= 1)

In [ ]:
train_df.shape

In [ ]:
test_df

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical


In [ ]:
from tqdm.notebook import tqdm
def extract(images):
    fet=[]
    for image in tqdm(images):
        img=load_img(image,color_mode='grayscale',target_size=(64,64))
        img=np.array(img)
        fet.append(img)
    fet=np.array(fet)
    features=fet.reshape(len(fet),64,64,1)
    
    return features

In [ ]:
image_path= "emotion\\Neutral\\imgres.html"
test_df = test_df[test_df['image'] != image_path]
test_df = test_df.reset_index(drop=True)


In [ ]:
train=extract(train_df['image'])

In [ ]:
test=extract(test_df['image'])

In [ ]:
train=train/255.0
test=test/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train_df['labels'])

In [ ]:
y_train=le.transform(train_df['labels'])
y_test=le.transform(test_df['labels'])

In [ ]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=train,y=y_train,batch_size=128,epochs=120,validation_data=(test,y_test))

In [ ]:
model_json=model.to_json()
with open("emotiondet.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondet.h5")

In [ ]:
from keras.models import model_from_json
json_file=open("emotiondet.json","r")
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("emotiondet.h5")

In [ ]:
labels=['Angry','Disguist','Fear','Happy','Neutral','Sad','Surprise']

In [ ]:
def check(image):
    img=load_img(image,color_mode='grayscale',target_size=(64,64))
    feature=np.array(img)
    feature=feature.reshape(1,64,64,1)
    return feature/255.0

In [ ]:
image="emotion/Angry/download (5).jpg"
print("original image is of Angry")
img=check(image)
pred=model.predict(img)
pred_label=labels[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(64,64),cmap='gray')

In [ ]:
image="emotion/Fear/download (2).jpg"
print("original image is of Fear")
img=check(image)
pred=model.predict(img)
pred_label=labels[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(64,64),cmap='gray')